# Collection
- Determines what the most recent lottery draw was
- Determines what the most recent lottery draw stored in the database is
- Collects the winning numbers, # of winners, total prize money for the recent draws not stored in the database
- Collects the stores' info where the 1st & 2nd place winners bought their tickets
- Collected data get stored into 'lottery.db' database.

The notebook is programmed to collect and store only the data that are currently not stored in the database but are present to be collected.<br>
For example, if the lottery's most recent draw was 4149th and we have stored until 4146th in the database, then this notebook collects & saves 4147th to 4149th draws into the database by running it.

## Setup

In [107]:
import sys
sys.path.append('..')

In [108]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from lib.progress_bar import progress_bar
import sqlite3

In [109]:
lotto_result_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin"
lotto_result_draw = lotto_result_url + "&drwNo="
store_url = "https://www.dhlottery.co.kr/store.do?method=topStore&pageGubun=L645&drwNo="

## Most Recent Draw
Let us determine what the most recent draw was in the real world.

In [110]:
def recentDraw():
    request = requests.get(lotto_result_url)
    soup = BeautifulSoup(request.text, "lxml")
    content = soup.find("meta", {"id" : "desc", "name" : "description"})['content']
    drawCount = re.compile(r'\d+회')
    mo = drawCount.search(content)
    return int(mo.group()[:-1])

In [111]:
mostRecent = recentDraw()
print(f"Most recent draw happend in the real world : {mostRecent}")

Most recent draw happend in the real world : 976


## Most Recent Draw in our Database
Let us now determine what the most recent draw stored in our database is.

In [112]:
db_path = "lottery.db"
sql_draw_create = """CREATE TABLE IF NOT EXISTS 
`DRAWS`(turn int, num_1 int, num_2 int, num_3 int, num_4 int, num_5 int, num_6 int, num_bonus int, UNIQUE(turn))"""

In [113]:
def recentDrawInDB(db_path):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute(sql_draw_create)
    cur.execute("SELECT MAX(turn) FROM DRAWS")
    info = cur.fetchall()
    con.commit()
    con.close()
    return int(info[0][0]) if info[0][0] else 0

In [114]:
dbMostRecent = recentDrawInDB(db_path)
print(f"Most recent draw stored in our database : {dbMostRecent}")

Most recent draw stored in our database : 0


## Draws Data Collection
The function below crawls over the Korean 6/45 lottery web html files and collects:
- Winning Numbers (1st, 2nd, ... 6th) + Bonus #
- \# of Winners (1st Place, ... 5th Place)
- Total Prize Money (1st Place, ... 5th Place)

In [115]:
def collectDraws(lst, start=dbMostRecent + 1, end=mostRecent):
    if start > end:
        print(f"No new draw to be added.")
        return
    print(f"Collecting Draws from Draw #{start} to #{end}")
    for idx in range(start, end + 1):
        progress_bar(idx - start, end - start + 1)
        
        draw = {'draw': idx}

        req = requests.get(lotto_result_draw + str(idx))
        soup = BeautifulSoup(req.text, 'lxml')

        meta = soup.find("meta", {"id" : "desc", "name" : "description"})['content']
        first, last = re.compile(r'(\d+,){4}\d+'), re.compile('\d+\+\d+')
        first_mo, last_mo = first.search(meta), last.search(meta)
        draw['nums'] = list(map(int, first_mo.group().split(',') + last_mo.group().split('+')))
        
        date = meta = soup.find("p", {"class" : "desc"})
        dateReg = re.compile(r'(\d){4}년 (\d){2}월 (\d){2}')
        mo = dateReg.search(str(date))
        draw['date'] = [int(mo.group()[:4]), int(mo.group()[6:8]), int(mo.group()[-2:])]

        total, winners = [], [0] * 5
        table = soup.find("table").find_all("td", {"class":"tar"})
        regex = re.compile(r'>\d+.*원')
        for i, c in enumerate(table):
            mo = regex.search(str(c))
            num = mo.group()[1:-1]
            num = int(num.replace(",", ""))
            if i % 2 == 0: total.append(num)
            elif num > 0: winners[i // 2] = total[-1] // num
        
        draw['winners'] = winners
        draw['prize'] = total
        
        lst.append(draw)
    progress_bar(end, end)
    
    print("")
    print("Done Collecting Draws!")

In [116]:
draws = []
collectDraws(lst=draws)

[====================================================================================================] 100%
Done Collecting Draws!


In [117]:
if len(draws) > 0:
    print(f"We have collected {len(draws)} draws")
    print(f"First draw : {draws[0]}")
    print(f"Last draw : {draws[-1]}")
else:
    print("No additional draw collected.")

We have collected 976 draws
First draw : {'draw': 1, 'nums': [10, 23, 29, 33, 37, 40, 16], 'date': [2002, 12, 7], 'winners': [0, 1, 28, 2537, 40155], 'prize': [0, 143934100, 143934000, 287695800, 401550000]}
Last draw : {'draw': 976, 'nums': [4, 12, 14, 25, 35, 37, 2], 'date': [2021, 8, 14], 'winners': [7, 69, 3085, 142375, 2270249], 'prize': [22707070505, 3784511793, 3784514495, 7118750000, 11351245000]}


## Stores Data Collection
The functions below crawl over the Korean 6/45 lottery web html files and collect:
- Stores' names (1st & 2nd Place Winners)
- Stores' addresses (1st & 2nd Place Winners)
- Whether the winning # were chosen automatic or manual (1st Place Winners Only)

In [118]:
def parseStores(content, win = 1):
    stores = []
    rows = content.find_all("tr")
    reg, regGen = re.compile(r'>.*<'), re.compile(r'[가-힣]+')

    for r in rows:
      tds = r.find_all('td')[1:4]
      if win == 2: tds = tds[:-1]
      store = []
      for i, td in enumerate(tds):
        if win == 1 and i == 1: td = str(regGen.search(str(td)).group())
        else: td = str(reg.search(str(td)).group()[1:-1])
        td = td.strip()
        store.append(td)
      stores.append(store)
    return stores

In [119]:
sql_store_create = """CREATE TABLE IF NOT EXISTS 
`STORES`(idx int, turn int, name varchar(255), auto BOOLEAN, firstPrize BOOLEAN, address varchar(255), UNIQUE(idx))"""

In [120]:
def dbStoresMostRecent(db_path):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute(sql_store_create)
    cur.execute("SELECT MAX(turn) FROM STORES")
    info = cur.fetchall()
    con.commit()
    con.close()
    return int(info[0][0]) if info[0][0] else 923

In [121]:
dbStoresMostRecent = dbStoresMostRecent(db_path)

In [122]:
##### 924회부터 제공
def collectLocations(lst, start=dbStoresMostRecent + 1, end=mostRecent):
    if start > end:
        print(f"No new draws to be added.")
        return
    if start < 924:
        print("Stores information is available since 924th draw.")
        return
    print(f"Collecting Locations from Draw #{start} to #{end}")

    table_class = "tbl_data tbl_data_col"

    for idx in range(start, end + 1):
        progress_bar(idx - start, end - start + 1)

        locations = {'draw': idx}
        idx_url = store_url + str(idx)

        soup = BeautifulSoup(requests.get(idx_url).text, 'lxml')
        tables = soup.find_all("table", {"class":table_class})
        tables = [x.find('tbody') for x in tables]

        locations['first'] = parseStores(content = tables[0])
        secondLocations = parseStores(content = tables[1], win = 2)

        maxPage = len(soup.find('div', {"id": "page_box"}).find_all('a'))
        for p in range(2, maxPage + 1):
            soup = BeautifulSoup(requests.get(idx_url + "&nowPage=" + str(p)).text, 'lxml')
            tables = soup.find_all("table", {"class":table_class})
            tables = [x.find('tbody') if i == 1 else None for i, x in enumerate(tables)]
            secondLocations.extend(parseStores(content = tables[1], win = 2))
        locations['second'] = secondLocations
        lst.append(locations)
    progress_bar(end, end)
    print("")
    print("Done Collecting Locations!")

In [123]:
stores = []
collectLocations(lst=stores)

[====================================================================================================] 100%
Done Collecting Locations!


In [124]:
if len(stores) > 0:
    print(f"We have collected {len(stores)} draws")
    print(f"First Set : {stores[0]}")
    print(f"Last Set : {stores[-1]}")
else:
    print("No additional stores collected.")

We have collected 53 draws
First Set : {'draw': 924, 'first': [['복권백화점', '자동', '서울 마포구 월드컵로 157'], ['CU(초읍대공원점)', '자동', '부산 부산진구 성지로 160 목화빌딩'], ['이마트24 광안리굿-7', '수동', '부산 수영구 광안로 49'], ['우리들공업탑점', '자동', '울산 남구 수암로 11'], ['지화자', '자동', '경기 성남시 수정구 위례광장로 328 1층 116호(창곡동, 우성위례타워)'], ['복권백화점', '자동', '경기 파주시 평화로 70'], ['행운복권방', '자동', '경기 포천시 솔모루로 86-1'], ['로또복권방', '자동', '충남 당진시 반촌로 199'], ['로또명당인주점', '자동', '충남 아산시 서해로 519-2']], 'second': [['씨유역삼에클라트로또판매점', '서울 강남구 테헤란로20길 19 엘지에클라트 1층 씨유역삼에클라트점'], ['로또카페', '서울 강서구 강서로74길 3 가양빌딩 1층 6호'], ['로또복권', '서울 노원구 상계동 649-7번지 다모아빌딩103'], ['스파', '서울 노원구 동일로 1493 상계주공아파트(10단지) 주공10단지종합상가111'], ['웨이스탑(전농점)', '서울 동대문구 전농로 129,(전농동)'], ['연초2호 쇼케이스7호', '서울 서초구 신반포로 194 강남고속버스터미널 쇼케이스7호'], ['인터넷 복권판매사이트', '동행복권(dhlottery.co.kr)'], ['인터넷 복권판매사이트', '동행복권(dhlottery.co.kr)'], ['인터넷 복권판매사이트', '동행복권(dhlottery.co.kr)'], ['인터넷 복권판매사이트', '동행복권(dhlottery.co.kr)'], ['버스카드충전소', '서울 서초구 신반포로 205 반포쇼핑타운 6동쇼핑앞'], ['훼미리24', '서울 성동구 아차산로11길 26'], ['잠실매점', '서울 송파구 올림픽로 269 잠실

## Raw Data to the Database
Insert each draws' data and stores' data to 'DRAWS' & 'STORES' tables in lottery.db'.

In [125]:
sql_draw = """INSERT INTO `DRAWS`(`turn`, `num_1`, `num_2`, `num_3`, `num_4`, `num_5`, `num_6`, `num_bonus`)
VALUES(?, ?, ?, ?, ?, ?, ?, ?)"""
sql_winner_create = """CREATE TABLE IF NOT EXISTS 
`WINNERS`(turn int, winner_1 int, winner_2 int, winner_3 int, winner_4 int, winner_5 int, UNIQUE(turn))"""
sql_winner = """INSERT INTO `WINNERS`(`turn`, `winner_1`, `winner_2`, `winner_3`, `winner_4`, `winner_5`)
VALUES(?, ?, ?, ?, ?, ?)"""
sql_prize_create = """CREATE TABLE IF NOT EXISTS 
`PRIZES`(turn int, prize_1 int, prize_2 int, prize_3 int, prize_4 int, prize_5 int, UNIQUE(turn))"""
sql_prize = """INSERT INTO `PRIZES`(`turn`, `prize_1`, `prize_2`, `prize_3`, `prize_4`, `prize_5`)
VALUES(?, ?, ?, ?, ?, ?)"""
sql_date_create = """CREATE TABLE IF NOT EXISTS `DATES`(turn int, year int, month int, day int, UNIQUE(turn))"""
sql_date = """INSERT INTO `DATES`(`turn`, `year`, `month`, `day`) VALUES(?, ?, ?, ?)"""
sql_store = """INSERT INTO `STORES`(`idx`, `turn`, `name`, `auto`, `firstPrize`, `address`) 
VALUES(?, ?, ?, ?, ?, ?)"""

In [126]:
def rawToDB(draws, stores, db_path):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    if len(draws) < 1 or len(stores) < 1:
        print("No additional data to be added to the database.")
        return

    cur.execute(sql_draw_create)
    cur.execute(sql_winner_create)
    cur.execute(sql_prize_create)
    cur.execute(sql_date_create)
    cur.execute(sql_store_create)

    for draw_info in draws:
        draw = draw_info["draw"]
        nums = draw_info["nums"]
        winners = draw_info["winners"]
        prizes = draw_info["prize"]
        dates = draw_info["date"]

        data = tuple([draw] + [int(n) for n in nums])
        cur.execute(sql_draw, data)
        data = tuple([draw] + [int(n) for n in winners])
        cur.execute(sql_winner, data)
        data = tuple([draw] + [int(n) for n in prizes])
        cur.execute(sql_prize, data)
        data = tuple([draw] + [int(n) for n in dates])
        cur.execute(sql_date, data)
        
    row_idx = 0
    for stores_info in stores:
        draw = stores_info["draw"] 
        for i in stores_info["first"]:
            data = tuple([row_idx, draw, str(i[0]), \
                1 if str(i[1]).strip() == "자동" else 0 
                , 1, str(i[-1])])
            cur.execute(sql_store, data)
            row_idx += 1
        for i in stores_info["second"]:
            data = tuple([row_idx, draw, str(i[0]), None, 0, str(i[-1])])
            cur.execute(sql_store, data)
            row_idx += 1
    con.commit()
    con.close()

In [127]:
rawToDB(draws, stores, db_path)